<a href="https://colab.research.google.com/github/TinaKhatri28/ASTRA-Project/blob/main/model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls /content/drive/MyDrive


In [ ]:

!ls "/content/drive/MyDrive/sih project"


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np

In [ ]:
DATA_PATH = "/content/drive/MyDrive/sih project/cotton crop"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 5  # Initial training
FINE_TUNE_EPOCHS = 10

In [ ]:

train_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.20,
    rotation_range=20,
    zoom_range=0.15,
    horizontal_flip=True
)
val_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.20
)

train_data = train_gen.flow_from_directory(
    DATA_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training",
    color_mode="rgb"
)
val_data = val_gen.flow_from_directory(
    DATA_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation",
    color_mode="rgb"
)

print("Class Mapping:", train_data.class_indices)


In [ ]:
base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.trainable = False  # Initial training: freeze all layers


model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])


In [ ]:
callbacks = [
    ModelCheckpoint("model1_best.h5", save_best_only=True, monitor="val_accuracy", mode="max"),
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.3, patience=3)
]

In [ ]:
model.compile(
    optimizer=Adam(5e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [ ]:


history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    callbacks=callbacks
)


In [ ]:
base_model.trainable = True
for layer in base_model.layers[:100]:  # Freeze first 100 layers
    layer.trainable = False

# Compile with lower learning rate
model.compile(
    optimizer=Adam(1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# Compute class weights
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_data.classes),
    y=train_data.classes
)
class_weights_dict = dict(enumerate(class_weights))


In [ ]:
history_fine = model.fit(
    train_data,
    validation_data=val_data,
    epochs=FINE_TUNE_EPOCHS,
    class_weight=class_weights_dict,
    callbacks=callbacks
)




In [ ]:
model.save("model1_final.h5")

In [ ]:
val_data.reset()
preds = model.predict(val_data, verbose=1)
pred_labels = (preds > 0.5).astype(int)
true_labels = val_data.classes

acc = accuracy_score(true_labels, pred_labels)
print(f"Validation Accuracy: {acc:.4f}")

cm = confusion_matrix(true_labels, pred_labels)
print("Confusion Matrix:")
print(cm)

report = classification_report(true_labels, pred_labels, target_names=list(val_data.class_indices.keys()))
print("Classification Report:")
print(report)

In [ ]:
model.save("/content/drive/MyDrive/sih_project/model1_final.h5")
